<a href="https://colab.research.google.com/github/ShinAsakawa/ShinAsakawa.github.io/blob/master/2025notebooks/2025_0626psylex71_CDP%2Bja1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="https://raw.githubusercontent.com/project-ccap/project-ccap.github.io/refs/heads/master/2025figs/1998Zorzi_CDP_fig1.svg" style="width:49%;"><br/>
<p>Zorzi+(1998) Fig.1 Architecture of the model. The arrow means full connectivity between layers. Each box stand for a group of letters (26) or phonemes (44).</p>


<img src="https://raw.githubusercontent.com/project-ccap/project-ccap.github.io/refs/heads/master/2025figs/1998Zorzi_CDP_fig8.svg" width="49%;"><br/>
<p>Zorzi+(1998) Fig.8. Architecture of the model with the hidden layer pathway. In both the direct pathway and the mediated pathway the layers are fully connected (arrows).</p>

<img src="https://raw.githubusercontent.com/project-ccap/project-ccap.github.io/refs/heads/master/2025figs/1998Zorzi_fig10.svg" width="49%"><br/>
<p style="align-text:center">
Figure 10. Lexical and sublexical procedures in reading aloud, and their interaction in the phonological decision system, where the final phonological code is computed for articulation.
</p>


# 0. 準備作業

In [ ]:
%config InlineBackend.figure_format = 'retina'
import torch
#device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device = torch.device('cuda:0' if torch.cuda.is_available() else 'mps' if torch.backends.mps.is_available() else 'cpu')
print(f'device:{device}')

# 必要なライブラリの輸入
from collections import OrderedDict
import sys
import os
import numpy as np
import operator
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt

HOME = os.environ['HOME']

from IPython import get_ipython
isColab =  'google.colab' in str(get_ipython())

try:
    import ipynbname
except ImportError:
    !pip install ipynbname
    import ipynbname

FILEPATH = str(ipynbname.path()).split('/')[-1]
print(f'FILEPATH:{FILEPATH}')

try:
    import japanize_matplotlib
except ImportError:
    !pip install japanize_matplotlib
    import japanize_matplotlib

## 0.1 モーラ tokenizer の定義 (モーラ分かち書き)

In [ ]:
# モーラ分かち書きの定義 source https://qiita.com/shimajiroxyz/items/a133d990df2bc3affc12
import re

# # 各条件を正規表現で表す
# c1 = '[ウクスツヌフムユルグズヅブプヴ][ァィェォ]' #ウ段＋「ァ/ィ/ェ/ォ」
# c2 = '[イキシチニヒミリギジヂビピ][ャュェョ]' #イ段（「イ」を除く）＋「ャ/ュ/ェ/ョ」
# c3 = '[テデ][ィュ]' #「テ/デ」＋「ャ/ィ/ュ/ョ」
# c4 = '[ァ-ヴー]' #カタカナ１文字（長音含む）
#
# cond = '('+c1+'|'+c2+'|'+c3+'|'+c4+')'
# re_mora = re.compile(cond)
#
# def moraWakachi(kana_text):
#     return re_mora.findall(kana_text)


class mora_Tokenizer:
    def __init__(self):
        special_tokens = ['<PAD>', '<EOW>', '<SOW>', '<UNK>']

        mora_list = ['ァ', 'ア', 'ィ', 'イ', 'イェ', 'ゥ', 'ウ', 'ウィ', 'ウェ', 'ウォ', 'ェ', 'エ', 'ォ', 'オ', 'カ', 'ガ', 'キ', 'キャ', 'キュ', 'キョ', 'ギ', 'ギャ', 'ギュ', 'ギョ', 'ク', 'クァ', 'クィ', 'クェ', 'クォ', 'グ', 'グァ', 'ケ', 'ゲ', 'コ', 'ゴ', 'サ', 'ザ', 'シ', 'シェ', 'シャ', 'シュ', 'ショ', 'ジ', 'ジェ', 'ジャ', 'ジュ', 'ジョ', 'ス', 'ズ', 'ズィ', 'セ', 'ゼ', 'ソ', 'ゾ', 'タ', 'ダ', 'チ', 'チェ', 'チャ', 'チュ', 'チョ', 'ヂ', 'ヂャ', 'ヂュ', 'ヂョ', 'ッ', 'ツ', 'ツァ', 'ツィ', 'ツェ', 'ツォ', 'ヅ', 'テ', 'ティ', 'テュ', 'デ', 'ディ', 'デュ', 'ト', 'ド', 'ナ', 'ニ', 'ニェ', 'ニャ', 'ニュ', 'ニョ', 'ヌ', 'ネ', 'ノ', 'ハ', 'バ', 'パ', 'ヒ', 'ヒェ', 'ヒャ', 'ヒュ', 'ヒョ', 'ビ', 'ビャ', 'ビュ', 'ビョ', 'ピ', 'ピャ', 'ピュ', 'ピョ', 'フ', 'ファ', 'フィ', 'フェ', 'フォ', 'ブ', 'ブィ', 'プ', 'ヘ', 'ベ', 'ペ', 'ホ', 'ボ', 'ポ', 'マ', 'ミ', 'ミャ', 'ミュ', 'ミョ', 'ム', 'メ', 'モ', 'ヤ', 'ュ', 'ユ', 'ョ', 'ヨ', 'ラ', 'リ', 'リェ', 'リャ', 'リュ', 'リョ', 'ル', 'レ', 'ロ', 'ヮ', 'ワ', 'ヲ', 'ン', 'ヴ', 'ヴァ', 'ヴィ', 'ヴェ', 'ヴォ', 'ー']

        self.tokens = special_tokens + mora_list

        # 各条件を正規表現で表す
        c1 = '[ウクスツヌフムユルグズヅブプヴ][ァィェォ]' #ウ段＋「ァ/ィ/ェ/ォ」
        c2 = '[イキシチニヒミリギジヂビピ][ャュェョ]' #イ段（「イ」を除く）＋「ャ/ュ/ェ/ョ」
        c3 = '[テデ][ィュ]' #「テ/デ」＋「ャ/ィ/ュ/ョ」
        c4 = '[ァ-ヴー]' #カタカナ１文字（長音含む）

        self.cond = '('+c1+'|'+c2+'|'+c3+'|'+c4+')'
        self.re_mora = re.compile(self.cond)

    def moraWakachi(self, kana_text):
        return self.re_mora.findall(kana_text)


    def wakachi(self, kana_text):
        kana_text = kana_text.replace('ヱ','エ').replace('ヰ','イ')
        morae = self.moraWakachi(kana_text)
        return morae

    def encode(self, kana_text):
        kana_text = kana_text.replace('ヱ','エ').replace('ヰ','イ')
        morae = self.moraWakachi(kana_text)
        ids = [self.tokens.index(_mora) for _mora in morae]
        return ids

    def decode(self, ids):
        out = []
        for idx in ids:
            m = self.tokens[idx]
            out.append(m)
        return out

    def __call__(self, kana_text):
        return self.encode(kana_text)

mora_tokenizer = mora_Tokenizer()
# mora_tokenizer の検証
# for _w in ['キチジョージ', 'チキン', 'ガッッキューホーカイー']:
#     print(_w, mora_tokenizer(_w), mora_tokenizer.decode(mora_tokenizer(_w)))
#     print(_w, mora_tokenizer.encode(_w))

## 0.2 訓令式ローマ字 tokenizer の定義

```
ア イ ウ エ オ     a i u e o              
カ キ ク ケ コ  キャ キュ キョ ka ki ku ke ko  kya kyu kyo          
サ シ ス セ ソ  シャ シュ ショ sa shi su se so  sha shu sho          
タ チ ツ テ ト  チャ チュ チョ ta chi tsu te to  cha chu cho          
ナ ニ ヌ ネ ノ  ニャ ニュ ニョ na ni nu ne no  nya nyu nyo          
ハ ヒ フ へ ホ  ヒャ ヒュ ヒョ ha hi fu he ho  hya hyu hyo          
マ ミ ム メ モ  ミャ ミュ ミョ ma mi mu me mo  mya myu myo          
ヤ  ユ  ヨ     ya  yu  yo              
ラ リ ル レ ロ  リャ リュ リョ ra ri ru re ro  rya  ryu ryo          
ワ    ヲ     wa    o              
ガ ギ グ ゲ ゴ  ギャ ギュ ギョ ga gi gu ge go  gya gyu gyo          
ザ ジ ズ ゼ ゾ  ジャ ジュ ジョ za ji zu ze zo  ja ju jo          
ダ ヂ ヅ デ ド  ヂャ ヂュ ヂョ da ji zu de do  ja ju jo          
バ ビ ブ ベ ボ  ビャ ビュ ビョ ba bi bu be bo  bya byu byo          
パ ピ プ ペ ポ  ピャ ピュ ピョ pa pi pu pe po  pya pyu pyo
```


In [ ]:
# 訓令式ローマ字トークナイザ kunrei_Tokenizer() の定義

# %load_ext autoreload
# %autoreload 2
# from kunrei import kunrei
#print(dir(kunrei)) # ('キチジョージ').split(' ')
#kunrei('ドォーモ')
#kunrei('ドォモ')
#kunrei('ドォ')    out_str = out_str.replace('プゥ', ' p u:')

# print(list(sorted(mora_dict.keys())))
# for m in sorted(list(sorted(mora_dict.keys()))):
#     print(f'm:{m}, kurei({m}).split(" "):{kunrei(m).split(" ")}')

class kunrei_Tokenizer():

    def __init__(self):

        self.kunrei_trans_dict = {
            'ァ':'a',    'ア':'a',    'ィ':'i',    'イ':'i',    'イェ': 'i e',
            'ゥ':'u',    'ウ':'u',    'ウィ':'u i',    'ウェ':'u e',    'ウォ':'u o',
            'ェ':'e',    'エ':'e',    'ォ':'o',    'オ':'o',    'カ':'k a',
            'ガ':'g a',    'キ':'k i',    'キャ':'ky a',    'キュ':'ky u',    'キョ':'ky o',
            'ギ':'g i',     'ギャ':'gy a',     'ギュ':'gy u',     'ギョ':'gy o',     'ク':'k u',
            'クァ':'k u a',     'クィ':'k u i',     'クェ':'k u e',     'クォ':'k u o',     'グ':'g u',
            'グァ':'g u a',     'ケ':'k e',     'ゲ':'g e',     'コ':'k o',     'ゴ':'g o',
            'サ':'s a',     'ザ':'z a',     'シ':'s a',     'シェ':'sy e',     'シャ':'sy a',
            'シュ':'sy u',     'ショ':'sy o',     'ジ':'g i',     'ジェ':'gy e',     'ジャ':'gy a',
            'ジュ':'gy u',     'ジョ':'gy o',     'ス':'s u',     'ズ':'z u',     'ズィ':'z i',
            'セ':'s e',     'ゼ':'z e',     'ソ':'s o',     'ゾ':'z o',     'タ':'t a',
            'ダ':'d a',     'チ':'t i',     'チェ':'ch e',     'チャ':'ch a',     'チュ':'ch u',
            'チョ':'ch o',     'ヂ':'z i',     'ヂャ':'zy a',     'ヂュ':'zy u',     'ヂョ':'zy o',
            'ッ':'t u',    'ツ':'t u',    'ツァ':'ty a',    'ツィ':'ty i',    'ツェ':'ty e',
            'ツォ':'ty o',     'ヅ':'z u',     'テ':'t e',     'ティ':'t i',     'テュ':'t u',
            'デ':'d e',     'ディ':'d i',     'デュ':'d u',     'ト':'t o',     'ド':'d o',
            'ナ':'n a',     'ニ':'n i',     'ニェ':'ny e',    'ニャ':'ny a',    'ニュ':'ny u',
            'ニョ':'ny o',    'ヌ':'n u',    'ネ':'n e',    'ノ':'n o',
            'ハ':'h a',    'バ':'b a',    'パ':'p a',    'ヒ':'h i',    'ヒェ':'hy e',
            'ヒャ':'hy a',    'ヒュ':'hy u',    'ヒョ':'hy o',    'ビ':'b i',    'ビャ':'by a',
            'ビュ':'by u',    'ビョ':'by o',    'ピ':'p i',    'ピャ':'py a',    'ピュ':'py u',
            'ピョ':'py o',    'フ':'f u',    'ファ':'f a',    'フィ':'f u i',    'フェ':'f u e',
            'フォ':'f u o',    'ブ':'b u',    'ブィ':'b i',    'プ':'p u',    'ヘ':'h e',
            'ベ':'b e',    'ペ':'p e',    'ホ':'h o',    'ボ':'b o',    'ポ':'p o',    'マ':'m a',
            'ミ':'m i',    'ミャ':'my a',    'ミュ':'my u',    'ミョ':'my o',
            'ム':'m u',    'メ':'m e',    'モ':'m o',    'ヤ':'y a',    'ュ':'y u',
            'ユ':'y o',    'ョ':'y o',    'ヨ':'y o',    'ラ':'r a',    'リ':'r i',
            'リェ':'ry e',    'リャ':'ry a',    'リュ':'ry u',    'リョ':'ry o',    'ル':'r u',
            'レ':'r e',    'ロ':'r o',    'ヮ':'w a',    'ワ':'w a',    'ヲ':'o',    'ン':'N',
            'ヴ':'b o',    'ヴァ':'b a',    'ヴィ':'b i',    'ヴェ':'b o',    'ヴォ':'b o',
            'ー':':', '〜':':'}
        self.mora_tokenizer = mora_Tokenizer()

        tokens = [':', 'N', 'a', 'b', 'by', 'ch', 'd', 'e', 'f', 'g', 'gy', 'h', 'hy', 'i', 'k', 'ky', 'm', 'my', 'n', 'ny', 'o', 'p', 'py', 'r', 'ry', 's', 'sy', 't', 'ty', 'u', 'w', 'y', 'z', 'zy']
        special_tokens = ['<PAD>', '<EOW>', '<SOW>', '<UNK>']
        self.tokens = special_tokens + tokens

    def encode(self, kana_text):
        # kana_text = kana_text.replace('ヱ','エ').replace('ヰ','イ')
        # mora_wakachi = self.mora_tokenizer.moraWakachi(kana_text)

        # phon = []
        # for mora in mora_wakachi:
        #     if not mora in self.kunrei_trans_dict:
        #         p = self.tokens.index('<UNK>')
        #     else:
        #         p = self.kunrei_trans_dict[mora].split(' ')
        #     phon = phon + p

        phon = self.wakachi(kana_text)
        out = []
        for p in phon:
            if not p in self.tokens:
                out.append(self.tokens.index('<UNK>'))
            else:
                out.append(self.tokens.index(p))
        return out

    def wakachi(self, kana_text):
        kana_text = kana_text.replace('ヱ','エ').replace('ヰ','イ')
        mora_wakachi = self.mora_tokenizer.moraWakachi(kana_text)

        phon = []
        for mora in mora_wakachi:
            if not mora in self.kunrei_trans_dict:
                p = self.tokens.index('<UNK>')
            else:
                p = self.kunrei_trans_dict[mora].split(' ')
            phon = phon + p
        return phon

    def decode(self, ids):
        out = []
        for idx in ids:
            m = self.tokens[idx]
            out.append(m)
        return out

    def __call__(self, kana_text):
        return self.encode(kana_text)

kunrei_tokenizer = kunrei_Tokenizer()
word = 'アカサタナ'
ids = kunrei_tokenizer(word)
print(word, ids, kunrei_tokenizer.decode(ids))
print(kunrei_tokenizer.wakachi(word))

## 0.3 文字の定義，(学習漢字，かな，カナ，数字，記号等)

In [ ]:
# 書記素の定義，書記素のうちカタカナを音韻表現としても利用

seed = 42
special_tokens = ['<PAD>', '<EOW>', '<SOW>', '<UNK>']
alphabet_upper_chars='ＡＢＣＤＥＦＧＨＩＪＫＬＭＮＯＰＱＲＳＴＵＶＷＸＹＺ'
alphabet_lower_chars='ａｂｃｄｅｆｇｈｉｊｋｌｍｎｏｐｑｒｓｔｕｖｗｘｙｚ'
num_chars='０１２３４５６７８９'
hira_chars='ぁあぃいぅうぇえぉおかがきぎくぐけげこごさざしじすずせぜそぞただちぢっつづてでとどなにぬねのはばぱひびぴふぶぷへべぺほぼぽまみむめもゃやゅゆょよらりるれろゎわゐゑをん'
kata_chars='ァアィイゥウェエォオカガキギクグケゲコゴサザシジスズセゼソゾタダチヂッツヅテデトドナニヌネノハバパヒビピフブプヘベペホボポマミムメモャヤュユョヨラリルレロヮワヰヱヲンヴヵヶ'
#kata_chars=kata_chars+'一'  # カタカナ文字に伸ばし記号を加える
#phon_list = list(kata_chars+'一')

# # 句点コード
# from RAM.char_ja import kuten as kuten
# kuten_chars=kuten().chars

# # 常用漢字
# from RAM.char_ja import chars_joyo as chars_joyo
# joyo_chars = "".join([ch for ch in chars_joyo().char_list])

# 学習漢字 学年別
_gakushu_list = ['一右雨円王音下火花貝学気休玉金九空月犬見五口校左三山四子糸字耳七車手十出女小上森人水正生青石赤先千川早草足村大男竹中虫町天田土二日入年白八百文本名木目夕立力林六',
'引羽雲園遠黄何夏家科歌画会回海絵外角楽活間丸岩顔帰汽記弓牛魚京強教近兄形計元原言古戸午後語交光公工広考行高合国黒今才細作算姉市思止紙寺時自室社弱首秋週春書少場色食心新親図数星晴声西切雪線船前組走多太体台谷知地池茶昼朝長鳥直通弟店点電冬刀東当答頭同道読内南肉馬買売麦半番父風分聞米歩母方北妹毎万明鳴毛門夜野矢友曜用来理里話',
'悪安暗委意医育員飲院運泳駅横屋温化荷界開階寒感漢館岸期起客宮急球究級去橋業局曲銀区苦具君係軽決血研県庫湖向幸港号根祭坂皿仕使始指死詩歯事持次式実写者主取守酒受州拾終習集住重宿所暑助勝商昭消章乗植深申真神身進世整昔全想相送息速族他打対待代第題炭短談着柱注丁帳調追定庭笛鉄転登都度島投湯等豆動童農波配倍箱畑発反板悲皮美鼻筆氷表病秒品夫負部服福物平返勉放味命面問役薬油有由遊予様洋羊葉陽落流旅両緑礼列練路和',
'愛案以位囲胃衣印栄英塩央億加果課貨芽改械害街各覚完官管観関願喜器希旗機季紀議救求泣給挙漁競共協鏡極訓軍郡型径景芸欠結健建験固候功好康航告差最菜材昨刷察札殺参散産残司史士氏試児治辞失借種周祝順初唱松焼照省笑象賞信臣成清静席積折節説戦浅選然倉巣争側束続卒孫帯隊達単置仲貯兆腸低停底的典伝徒努灯働堂得特毒熱念敗梅博飯費飛必標票不付府副粉兵別変辺便包法望牧末満未脈民無約勇要養浴利陸料良量輪類令例冷歴連労老録',
'圧易移因営永衛液益演往応恩仮価可河過賀解快格確額刊幹慣眼基寄規技義逆久旧居許境興均禁句群経潔件券検険減現限個故護効厚構耕講鉱混査再妻採災際在罪財桜雑賛酸師志支枝資飼似示識質舎謝授修術述準序承招証常情条状織職制勢性政精製税績責接設絶舌銭祖素総像増造則測属損態貸退団断築張提程敵適統導銅徳独任燃能破判版犯比肥非備俵評貧婦富布武復複仏編弁保墓報豊暴貿防務夢迷綿輸余預容率略留領',
'異遺域宇映延沿我灰拡閣革割株巻干看簡危揮机貴疑吸供胸郷勤筋敬系警劇激穴憲権絹厳源呼己誤后孝皇紅鋼降刻穀骨困砂座済裁策冊蚕姿私至視詞誌磁射捨尺若樹収宗就衆従縦縮熟純処署諸除傷将障城蒸針仁垂推寸盛聖誠宣専泉洗染善創奏層操窓装臓蔵存尊宅担探誕暖段値宙忠著庁潮頂賃痛展党糖討届難乳認納脳派俳拝背肺班晩否批秘腹奮並閉陛片補暮宝訪亡忘棒枚幕密盟模訳優郵幼欲翌乱卵覧裏律臨朗論']

_l = []
for g in _gakushu_list:
    for ch in g:
        _l += ch
gakushu_chars = "".join(ch for ch in _l)

grph_list = []
#for x in [hira_chars]:                       # 数字は入力文字としない場合
#for x in [hira_chars, gakushu_chars]:             # 数字は入力文字としない場合
for x in [hira_chars, kata_chars, num_chars, gakushu_chars]: # 数字も入力文字とする場合
    for ch in x:
        grph_list.append(ch)
print(f'len(grph_list):{len(grph_list)}')
print(f'全書記素 grph_list:{"".join([ch for ch in grph_list])}')

# print(f'len(phon_list):{len(phon_list)}')
# print(f'全音素 phon_list:{phon_list}')

print(f'入力層の素子数 len(grph_list) + len(special_tokens)={len(grph_list) + len(special_tokens)}')
# print(f'出力層の素子数 len(phon_list) + len(special_tokens)={len(phon_list) + len(special_tokens)}')

## 0.4 学習文字 tokenizer の定義 gakushu_Tokenizer()

In [ ]:
class gakushu_Tokenizer():
    def __init__(self):
        special_tokens = ['<PAD>', '<EOW>', '<SOW>', '<UNK>']
        alphabet_upper_chars='ＡＢＣＤＥＦＧＨＩＪＫＬＭＮＯＰＱＲＳＴＵＶＷＸＹＺ'
        alphabet_lower_chars='ａｂｃｄｅｆｇｈｉｊｋｌｍｎｏｐｑｒｓｔｕｖｗｘｙｚ'
        num_chars='０１２３４５６７８９'
        hira_chars='ぁあぃいぅうぇえぉおかがきぎくぐけげこごさざしじすずせぜそぞただちぢっつづてでとどなにぬねのはばぱひびぴふぶぷへべぺほぼぽまみむめもゃやゅゆょよらりるれろゎわゐゑをん'
        kata_chars='ァアィイゥウェエォオカガキギクグケゲコゴサザシジスズセゼソゾタダチヂッツヅテデトドナニヌネノハバパヒビピフブプヘベペホボポマミムメモャヤュユョヨラリルレロヮワヰヱヲンヴヵヶ'

        # 学習漢字 学年別
        gakushu_chars = '一右雨円王音下火花貝学気休玉金九空月犬見五口校左三山四子糸字耳七車手十出女小上森人水正生青石赤先千川早草足村大男竹中虫町天田土二日入年白八百文本名木目夕立力林六' + '引羽雲園遠黄何夏家科歌画会回海絵外角楽活間丸岩顔帰汽記弓牛魚京強教近兄形計元原言古戸午後語交光公工広考行高合国黒今才細作算姉市思止紙寺時自室社弱首秋週春書少場色食心新親図数星晴声西切雪線船前組走多太体台谷知地池茶昼朝長鳥直通弟店点電冬刀東当答頭同道読内南肉馬買売麦半番父風分聞米歩母方北妹毎万明鳴毛門夜野矢友曜用来理里話' + '悪安暗委意医育員飲院運泳駅横屋温化荷界開階寒感漢館岸期起客宮急球究級去橋業局曲銀区苦具君係軽決血研県庫湖向幸港号根祭坂皿仕使始指死詩歯事持次式実写者主取守酒受州拾終習集住重宿所暑助勝商昭消章乗植深申真神身進世整昔全想相送息速族他打対待代第題炭短談着柱注丁帳調追定庭笛鉄転登都度島投湯等豆動童農波配倍箱畑発反板悲皮美鼻筆氷表病秒品夫負部服福物平返勉放味命面問役薬油有由遊予様洋羊葉陽落流旅両緑礼列練路和' + '愛案以位囲胃衣印栄英塩央億加果課貨芽改械害街各覚完官管観関願喜器希旗機季紀議救求泣給挙漁競共協鏡極訓軍郡型径景芸欠結健建験固候功好康航告差最菜材昨刷察札殺参散産残司史士氏試児治辞失借種周祝順初唱松焼照省笑象賞信臣成清静席積折節説戦浅選然倉巣争側束続卒孫帯隊達単置仲貯兆腸低停底的典伝徒努灯働堂得特毒熱念敗梅博飯費飛必標票不付府副粉兵別変辺便包法望牧末満未脈民無約勇要養浴利陸料良量輪類令例冷歴連労老録' + '圧易移因営永衛液益演往応恩仮価可河過賀解快格確額刊幹慣眼基寄規技義逆久旧居許境興均禁句群経潔件券検険減現限個故護効厚構耕講鉱混査再妻採災際在罪財桜雑賛酸師志支枝資飼似示識質舎謝授修術述準序承招証常情条状織職制勢性政精製税績責接設絶舌銭祖素総像増造則測属損態貸退団断築張提程敵適統導銅徳独任燃能破判版犯比肥非備俵評貧婦富布武復複仏編弁保墓報豊暴貿防務夢迷綿輸余預容率略留領' + '異遺域宇映延沿我灰拡閣革割株巻干看簡危揮机貴疑吸供胸郷勤筋敬系警劇激穴憲権絹厳源呼己誤后孝皇紅鋼降刻穀骨困砂座済裁策冊蚕姿私至視詞誌磁射捨尺若樹収宗就衆従縦縮熟純処署諸除傷将障城蒸針仁垂推寸盛聖誠宣専泉洗染善創奏層操窓装臓蔵存尊宅担探誕暖段値宙忠著庁潮頂賃痛展党糖討届難乳認納脳派俳拝背肺班晩否批秘腹奮並閉陛片補暮宝訪亡忘棒枚幕密盟模訳優郵幼欲翌乱卵覧裏律臨朗論'

        self.tokens = special_tokens + list(num_chars) + list(hira_chars) + list(kata_chars) + list(gakushu_chars)

    def encode(self, chars):
        out = []
        for ch in chars:
            if not ch in self.tokens:
                out.append(self.tokens.index('<UNK>'))
            else:
                out.append(self.tokens.index(ch))
        return out

    def decode(self, ids):
        out = [self.tokens[idx] for idx in ids]
        return out

    def __call__(self, chars):
        return self.encode(chars)

gakushu_tokenizer = gakushu_Tokenizer()
# 上記 gakushu_tokenizer の検証
# print(gakushu_tokenizer.char_list)
# #gakushu_tokenizer.encode('学校')
# print(gakushu_tokenizer('学校'))
# print(gakushu_tokenizer.decode(gakushu_tokenizer('学校')))
#print(len(gakushu_tokenizer.tokens), len(mora_tokenizer.tokens),)

## 0.5 NTT 日本語の語彙特性 単語頻度データの読み込み

In [ ]:
if isColab:
    !pip install googledrivedownloader==0.4
    from google_drive_downloader import GoogleDriveDownloader as gdd
    import os

    # 共有ファイルのIDを指定
    file_id = '1eBJDN392BsUckg5LBFbbw5KT9PCsmnxI' # 'psylex71utf8_.txt
    # https://drive.google.com/file/d/1eBJDN392BsUckg5LBFbbw5KT9PCsmnxI/view?usp=drive_link

    # 保存したい場所とファイル名を指定\n",
    # 例: /content/ ディレクトリに original_file_name.拡張子 という名前で保存\n",
    destination_path = '/content/psylex71utf8_.txt' # ファイルの拡張子を適切に設定してください\n",
    try:
        print(f"ファイルのダウンロードを開始します (ファイルID: {file_id})...")
        gdd.download_file_from_google_drive(file_id=file_id,
                                            dest_path=destination_path)
                                            # unzip=True if file_id is for a zip file):
        print(f"ファイルのダウンロードが完了しました。'{destination_path}' に保存されました。")

        # ダウンロードしたファイルを読み込む例 (テキストファイルの場合)
        if os.path.exists(destination_path):
            print("ダウンロードしたファイルの内容 (最初の数行):")
            with open(destination_path, 'r') as f:
                # ファイルの内容を表示 (例: 最初の5行)
                for i in range(5):
                    line = f.readline()
                    if not line:
                        break
                    print(line.strip())
        else:
            print(f"エラー: ダウンロード先のファイル '{destination_path}' が見つかりません。")

    except Exception as e:
        print(f"ファイルのダウンロード中にエラーが発生しました: {e}")

In [ ]:
# NTT 日本語の語彙特性単語頻度データ psylex71.txt の読み込み
#HOME = os.environ['HOME']
if isColab:
    ntt_base = '/content'
else:
    ntt_base = os.path.join(HOME, 'study/2017_2009AmanoKondo_NTTKanjiData')
psy71_fname = os.path.join(ntt_base, 'psylex71utf8_.txt')  # ファイル名
psylex71raw = open(psy71_fname, 'r').readlines()
psylex71raw = [lin.strip().split(' ')[:6] for lin in psylex71raw]   # 空白 ' ' で分離し，年度ごとの頻度を削除
print(f'len(psylex71raw):{len(psylex71raw)}')

valid_chars = kata_chars + 'ー〜'

# Psylex71 一行のデータは 0:共通ID, 1:独自ID, 2:表記, 3:ヨミ, 4:品詞, 5:頻度 を取り出す。
#n_idx=0; n_wrd=2; n_yomi=3; n_pos=4; n_frq=5
psylex_ids = {'_idx':0, '_idx2':1, '_wrd':2, '_yomi':3, '_pos':4, '_frq':5, '_mora':6}
psylex_ids['_kunrei'] = 7
print(f'psylex_ids{psylex_ids}')

mora_dict = OrderedDict()
kunrei_dict = OrderedDict()

for x in tqdm(psylex71raw[1:]):
    _word =  x[psylex_ids['_wrd']]
    _yomi = x[psylex_ids['_yomi']]
    is_valid = True
    for ch in _yomi:
        if not ch in valid_chars:
            is_valid = False
    if is_valid:
        morae = mora_tokenizer.wakachi(_yomi)
        #morae = moraWakachi(_yomi)
        for m in morae:
            if not m in mora_dict:
                mora_dict[m] = 1
            else:
                mora_dict[m] += 1

print(f'len(kunrei_dict):{len(kunrei_dict)}')
kunrei_list = sorted(kunrei_dict.keys())
print(f'kurei_list:{kunrei_list}')

print(f'len(mora_dict):{len(mora_dict)}')
mora_list = sorted(mora_dict.keys())
print(len(mora_dict), mora_dict)

is_graph = True
_dict = mora_dict
if is_graph:
    _N = np.array([v for v in _dict.values()]).sum()
    _count_sorted = sorted(_dict.items(), key=operator.itemgetter(1), reverse=True)
    figsize=(24,4)
    topN = 100 if _N > 100 else _N
    plt.figure(figsize=figsize)
    plt.bar(range(topN), [x[1]/_N for x in _count_sorted[:topN]])
    plt.xticks(ticks=range(topN), labels=[c[0] for c in _count_sorted[:topN]])

    plt.title(f'モーラ頻度 (上位:{topN} 語)')
    plt.ylabel('相対頻度')
    plt.show()
    #len(mora_dict)

# 1. データセットの定義

In [ ]:
maxlen_grph = 2        # 書記素最大文字数 + 2 しているのは, 単語の前後に特殊トークン <SOW> <EOW> をつけるため
valid_chars=grph_list  # 書記素リスト grph_list を有効文字リスト valid_chars とする
ng_yomi_words = []
dups_idx = []
_psylex71_ = []

Psylex71 = OrderedDict()
for lin in psylex71raw:
    wrd = lin[psylex＿ids['_wrd']]
    idx = lin[psylex＿ids['_idx']]
    yomi = lin[psylex＿ids['_yomi']]
    pos = lin[psylex＿ids['_pos']]
    frq = lin[psylex＿ids['_frq']]

    # print(f'type(lin):{type(lin)}')
    # print(f'lin:{lin}')
    # sys.exit()

    if len(wrd) == maxlen_grph:  # 長さが maxlen_grph 文字である語に対して処理を行う

        # ヨミの中にカタカナ以外の文字が入っていれば NG_flag を True にする
        is_kata_yomi = True
        for p in yomi:
            if not p in kata_chars:
                is_kata_yomi = False

        # ヨミにカタカナ以外の文字が含まれていれば ng_yomi_words に加える
        if is_kata_yomi == False:
            ng_yomi_words.append((wrd,yomi))
        else:

            # valid_chars (学習漢字+)で構成されているか否かを判断
            is_valid_grph = True
            for i in range(maxlen_grph):
                if not wrd[i] in valid_chars:
                    is_valid_grph = False

            if is_valid_grph == True:

                _mora = mora_tokenizer.wakachi(yomi) # .strip()  # モーラ分かち書きを行う
                _kunrei = kunrei_tokenizer.wakachi(yomi)
                if idx in Psylex71:   # すでに ID 番号が登録されていれば dups_idx リストに加える
                    dups_idx.append((idx, lin, (Psylex71[idx]['単語'], Psylex71[idx]['ヨミ'], _mora)))

                Psylex71[idx] = {'単語': wrd, 'モーラ':_mora, 'ヨミ': yomi,
                                 '訓令':_kunrei, '品詞': pos,'頻度': frq}

                _psylex71_.append(lin + [_mora, _kunrei])


# 読み (音韻表現) の最大長値の探索
maxlen_mora, maxlen_kunrei = 0, 0
for a in _psylex71_:
    if len(a[psylex_ids['_mora']]) > maxlen_mora:
         maxlen_mora = len(a[psylex_ids['_mora']])
    if len(a[psylex_ids['_kunrei']]) > maxlen_kunrei:
         maxlen_kunrei = len(a[psylex_ids['_kunrei']])

# 結果の表示
print(f'読み込んだ psylex71.txt の単語数 len(psylex71raw):{len(psylex71raw)}')
print(f'Psylex71 の総単語数 len(_psylex71_):{len(_psylex71_)}')
print(f'作成したデータベース辞書の項目数 len(Psylex71):{len(Psylex71)}')
print(f'ヨミの最長モーラ数 maxlen_mora:{maxlen_mora}')
print(f'訓令式の最長トークン数 maxlen_kunrei:{maxlen_kunrei}')
print(f'len(mora_list):{len(mora_list)}')
#print(f'音素 (読みのカタカナ文字)数 len(phon_cands):{len(phon_cands)}')
print(f'Psylex71 におけるカタカナ以外のヨミのある単語数 len(ng_yomi_words):{len(ng_yomi_words)}')
print(f'Psylex71 における ID 番号の重複数 len(dups_idx):{len(dups_idx)}')

# 2． Psylex71_Dataset (PyTorch 用のクラス) の作成

In [ ]:
import torch
class Psylex71_Dataset(torch.utils.data.Dataset):
    '''ニューラルネットワークモデルに Psylex71 を学習させるための PyTorch 用データセットのクラス'''

    def __init__(self,
                 dic=Psylex71,
                 input_tokenizer=gakushu_tokenizer,
                 output_tokenizer=mora_tokenizer,
                 special_tokens=special_tokens,
                 device=device):
        super().__init__()
        self.dic = dic
        self.inputs = [v['単語'] for v in dic.values()]
        self.targets = [v['ヨミ'] for v in dic.values()]
        self.special_tokens = special_tokens
        self.device = device

        self.input_tokenizer = input_tokenizer
        self.output_tokenizer = output_tokenizer

        maxlen_out = 0
        for k, v in dic.items():
            _len = len(self.output_tokenizer(v['ヨミ']))
            maxlen_out = _len if _len > maxlen_out else maxlen_out

        # ＋2 しているのは <SOW>,<EOW> という 2 つのスペシャルトークンを付加するため
        self.maxlen_out = maxlen_out + 2


    def __len__(self):
        return len(self.dic)

    def __getitem__(self, idx):
        inp, tgt = self.inputs[idx], self.targets[idx]

        # 入力信号にも <SOW>, <EOW> トークンを付与する場合
        #inp = [self.input_cands.index('<SOW>')]  + [self.input_cands.index(x) for x in inp]  + [self.input_cands.index('<EOW>')]

        # 入力信号にはスペシャルトークンを付与しない場合
        #inp = [self.input_tokenizer.index(x) for x in inp]
        inp = self.input_tokenizer(inp)

        # ターゲット (教師)信号 には <SOW>, <EOW> を付与する
        #tgt = [self.target_tokecands.index('<SOW>')] + [self.target_cands.index(x) for x in tgt] + [self.target_cands.index('<EOW>')]
        tgt = self.output_tokenizer(tgt)
        tgt = [self.output_tokenizer.tokens.index('<SOW>')] + tgt + [self.output_tokenizer.tokens.index('<EOW>')]

        while len(tgt) < self.maxlen_out:
            #tgt = tgt + [self.target_cands.index('<PAD>')]
            tgt = tgt + [self.output_tokenizer.tokens.index('<PAD>')]

        inp, tgt = torch.LongTensor(inp), torch.LongTensor(tgt)
        inp, tgt = inp.to(self.device), tgt.to(self.device)
        return inp, tgt

    def getitem(self, idx):
        #inp, tgt = self.inputs[idx], self.targets[idx]
        wrd = self.inputs[idx]
        phn = self.targets[idx]
        return wrd, phn

    def ids2argmax(self, ids):
        out = np.array([torch.argmax(idx).numpy() for idx in ids], dtype=np.int32)
        return out

    def ids2tgt(self, ids):
        # out = [self.target_cands[idx - len(self.special_tokens)] for idx in ids]
        out = self.output_tokenizer.decode(ids)
        return out

    def ids2inp(self, ids):
        #out = [self.input_cands[idx] for idx in ids]
        out = self.input_tokenizer.decode(ids)
        return out

    def target_ids2target(self, ids:list):
        ret = self.output_tokenizer.decode(ids)
        return ret

psylex71_ds = Psylex71_Dataset()

# 3. モデルの定義
## 3.1 TLA モデルの定義

In [ ]:
# 全モデル共通使用するライブラリの輸入
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.nn.utils.rnn import pad_sequence

In [ ]:
class vanilla_TLA(torch.nn.Module):
    def __init__(self,
                 inp_vocab_size:int=len(psylex71_ds.input_tokenizer.tokens),
                 out_vocab_size:int=len(psylex71_ds.output_tokenizer.tokens),
                 inp_len:int=maxlen_grph,
                 out_len:int=psylex71_ds.maxlen_out,
                 #out_len:int=maxlen_mora+2,
                 n_hid:int=1024,
                 device:str=device):

        super().__init__()
        self.inp_vocab_size=inp_vocab_size
        self.inp_len=inp_len
        self.out_vocab_size=out_vocab_size
        self.out_len=out_len
        self.n_hid=n_hid

        self.emb_layer = torch.nn.Linear(in_features=inp_vocab_size * inp_len, out_features=n_hid).to(device)
        #self.sigmoid = torch.nn.Sigmoid()
        #self.tanh = torch.nn.Tanh()
        #self.relu = torch.nn.ReLU()
        self.emb_outf = torch.nn.Tanh()
        self.out_outf = torch.nn.Sigmoid()
        #self.out_outf = torch.nn.Tanh()

        self.out_layer = torch.nn.Linear(in_features=n_hid, out_features=out_vocab_size * out_len).to(device)

    def forward(self, X, Y):
        '''互換性のため Y を入力としているが実際には使っていない'''

        # 入力 X はトークン ID リストであるので，ワンホットベクトル化する
        X = torch.nn.functional.one_hot(X, num_classes=self.inp_vocab_size)

        X = X.reshape(X.size(0),-1) # ワンホットベクトルを連接して行ベクトルに変換
        X = X.float()               # ワンホットベクトルは整数 int64 なので浮動小数点に変換

        X = self.emb_layer(X)       # 埋め込み層への信号伝搬
        X = self.emb_outf(X)        # 埋め込み層の非線形変換

        X = self.out_layer(X)       # 出力層への信号伝搬
        X = self.out_outf(X)        # 出力層での非線形変換

        # 各出力ニューロンに分割
        X = X.reshape(X.size(0), self.out_len, self.out_vocab_size)

        return X

vanilla_tla = vanilla_TLA(device=device)
print(vanilla_tla.eval())
print(f'vanilla_tla.out_len:{vanilla_tla.out_len}')

## 3.2 seq2seq_with_attention model の定義

In [ ]:
class Seq2Seq_wAtt(nn.Module):
    """ 注意つき符号化器‐復号化器モデル
    Bahdanau, Cho, & Bengio (2015) NEURAL MACHINE TRANSLATION BY JOINTLY LEARNING TO ALIGN AND TRANSLATE, arXiv:1409.0473
    """
    def __init__(self,
                 enc_vocab_size:int,
                 dec_vocab_size:int,
                 n_hid:int,
                 n_layers:int=2,
                 bidirectional:bool=False,
                 device=device):
        super().__init__()

        # Encoder 側の入力トークン id を多次元ベクトルに変換
        self.encoder_emb = nn.Embedding(num_embeddings=enc_vocab_size,
                                        embedding_dim=n_hid,
                                        padding_idx=0)

        # Decoder 側の入力トークン id を多次元ベクトルに変換
        self.decoder_emb = nn.Embedding(num_embeddings=dec_vocab_size,
                                        embedding_dim=n_hid,
                                        padding_idx=0)

        # Encoder LSTM 本体
        self.encoder = nn.LSTM(input_size=n_hid,
                               hidden_size=n_hid,
                               num_layers=n_layers,
                               batch_first=True,
                               bidirectional=bidirectional)

        # Decoder LSTM 本体
        self.decoder = nn.LSTM(input_size=n_hid,
                               hidden_size=n_hid,
                               num_layers=n_layers,
                               batch_first=True,
                               bidirectional=bidirectional)

        # 文脈ベクトルと出力ベクトルの合成を合成する層
        bi_fact = 2 if bidirectional else 1
        self.combine_layer = nn.Linear(bi_fact * 2 * n_hid, n_hid)

        # 最終出力層
        self.out_layer = nn.Linear(n_hid, dec_vocab_size)

    def forward(self, enc_inp, dec_inp):

        enc_emb = self.encoder_emb(enc_inp)
        enc_out, (hnx, cnx) = self.encoder(enc_emb)

        dec_emb = self.decoder_emb(dec_inp)
        dec_out, (hny, cny) = self.decoder(dec_emb,(hnx, cnx))

        # enc_out は (バッチサイズ，ソースの単語数，中間層の次元数)
        # ソース側 (enc_out) の各単語とターゲット側 (dec_out) の各単語との類似度を測定するため
        # 両テンソルの内積をとるため ソース側 (enc_out) の軸を入れ替え
        enc_outP = enc_out.permute(0,2,1)

        # sim の形状は (バッチサイズ, 中間層の次元数，ソースの単語数)
        sim = torch.bmm(dec_out, enc_outP)

        # sim の各次元のサイズを記録
        batch_size, dec_word_size, enc_word_size = sim.shape

        # sim に対して，ソフトマックスを行うため形状を変更
        simP = sim.reshape(batch_size * dec_word_size, enc_word_size)

        # simP のソフトマックスを用いて注意の重み alpha を算出
        alpha = F.softmax(simP,dim=1).reshape(batch_size, dec_word_size, enc_word_size)

        # 注意の重み alpha に encoder の出力を乗じて，文脈ベクトル c_t とする
        c_t = torch.bmm(alpha, enc_out)

        # torch.cat だから c_t と dec_out とで合成
        dec_out_ = torch.cat([c_t, dec_out], dim=2)
        dec_out_ = self.combine_layer(dec_out_)

        return self.out_layer(dec_out_)

    def evaluate(self, enc_inp, dec_inp):

        enc_emb = self.encoder_emb(enc_inp)
        enc_out, (hnx, cnx) = self.encoder(enc_emb)

        dec_emb = self.decoder_emb(dec_inp)
        dec_out, (hny, cny) = self.decoder(dec_emb,(hnx, cnx))
        return self.out_layer(dec_out)

# # 以下確認作業
# ds = train_ds
n_layers=1
bidirectional=False
n_hid=128
tla_seq2seq = Seq2Seq_wAtt(enc_vocab_size=len(gakushu_tokenizer.tokens),
                           dec_vocab_size=len(mora_tokenizer.tokens),
                           n_layers=n_layers,
                           bidirectional=bidirectional,
                           n_hid=n_hid).to(device)
print(tla_seq2seq.eval())

## 3.3 seq2seq_without_attention model の定義

In [ ]:
class Seq2Seq_woAtt(nn.Module):
    """ 注意つき符号化器‐復号化器モデル
    Bahdanau, Cho, & Bengio (2015) NEURAL MACHINE TRANSLATION BY JOINTLY LEARNING TO ALIGN AND TRANSLATE, arXiv:1409.0473
    """
    def __init__(self,
                 enc_vocab_size:int,
                 dec_vocab_size:int,
                 n_hid:int,
                 n_layers:int=2,
                 bidirectional:bool=False,
                 device=device):
        super().__init__()

        # Encoder 側の入力トークン id を多次元ベクトルに変換
        self.encoder_emb = nn.Embedding(num_embeddings=enc_vocab_size,
                                        embedding_dim=n_hid,
                                        padding_idx=0)

        # Decoder 側の入力トークン id を多次元ベクトルに変換
        self.decoder_emb = nn.Embedding(num_embeddings=dec_vocab_size,
                                        embedding_dim=n_hid,
                                        padding_idx=0)

        # Encoder LSTM 本体
        self.encoder = nn.LSTM(input_size=n_hid,
                               hidden_size=n_hid,
                               num_layers=n_layers,
                               batch_first=True,
                               bidirectional=bidirectional)

        # Decoder LSTM 本体
        self.decoder = nn.LSTM(input_size=n_hid,
                               hidden_size=n_hid,
                               num_layers=n_layers,
                               batch_first=True,
                               bidirectional=bidirectional)

        # 文脈ベクトルと出力ベクトルの合成を合成する層
        bi_fact = 2 if bidirectional else 1
        self.combine_layer = nn.Linear(bi_fact * 2 * n_hid, n_hid)

        # 最終出力層
        self.out_layer = nn.Linear(n_hid, dec_vocab_size)

    def forward(self, enc_inp, dec_inp):

        enc_emb = self.encoder_emb(enc_inp)
        enc_out, (hnx, cnx) = self.encoder(enc_emb)

        dec_emb = self.decoder_emb(dec_inp)
        dec_out, (hny, cny) = self.decoder(dec_emb,(hnx, cnx))

        return self.out_layer(dec_out)

    def evaluate(self, enc_inp, dec_inp):
        return self.forward(enc_inp, dec_inp)


# # 以下確認作業
# ds = train_ds
n_layers=1
bidirectional=False
n_hid=128
tla_seq2seq = Seq2Seq_wAtt(enc_vocab_size=len(gakushu_tokenizer.tokens),
                           dec_vocab_size=len(mora_tokenizer.tokens),
                           n_layers=n_layers,
                           bidirectional=bidirectional,
                           n_hid=n_hid).to(device)
print(tla_seq2seq.eval())

tla_seq2seq0 = Seq2Seq_woAtt(enc_vocab_size=len(gakushu_tokenizer.tokens),
                             dec_vocab_size=len(mora_tokenizer.tokens),
                             n_layers=n_layers,
                             bidirectional=bidirectional,
                             n_hid=n_hid).to(device)
print(tla_seq2seq0.eval())

# 4. 訓練 (train) データセット，検証 (valid) データセット，検査 (test) データセットへ分割

## 4.1 データセットの選択

In [ ]:
# 以下の 2 つのデータセットは出力用トークナイザによって2つに分かれる
psylex71_ds_mora = Psylex71_Dataset(output_tokenizer=mora_tokenizer)
psylex71_ds_kunrei = Psylex71_Dataset(output_tokenizer=kunrei_tokenizer)

# データセットのチェック
for _ds in [psylex71_ds_mora, psylex71_ds_kunrei]:
    for N in np.random.permutation(_ds.__len__())[:5]:
    #for N in range(3):
        inp, tgt = _ds.__getitem__(N)
        print(f'_ds.ids2inp(inp):{_ds.ids2inp(inp)}',
              f'{inp.cpu().numpy()}',
              f'_ds.target_ids2target(tgt):{_ds.target_ids2target(tgt)}',
              f'{tgt.cpu().numpy()}')

## 4.2 データセットの分割,訓練,検査,検証データセット

In [ ]:
# データセットの分割,訓練,検査,検証データセット
_ds = psylex71_ds_mora

train_size = int(_ds.__len__() * 0.5)
valid_size = int(_ds.__len__() * 0.1)
resid_size = _ds.__len__() - train_size - valid_size
train_ds, valid_ds, resid_size = torch.utils.data.random_split(
    dataset=_ds,
    lengths=(train_size, valid_size, resid_size),
    generator=torch.Generator().manual_seed(seed))

## 4.3 バッチサイズの定義とデータローダの設定

In [ ]:
# batch_size = 32
# batch_size = 64
# batch_size = 4096
batch_size = 128
#batch_size = 512
train_dl = torch.utils.data.DataLoader(dataset=train_ds, batch_size=batch_size, shuffle=True)
valid_dl = torch.utils.data.DataLoader(dataset=valid_ds, batch_size=batch_size, shuffle=False)

def _collate_fn(batch):
    inps, tgts = list(zip(*batch))
    inps = list(inps)
    tgts = list(tgts)
    return inps, tgts

train_dl = torch.utils.data.DataLoader(
    dataset=train_ds,
    batch_size=batch_size,
    shuffle=True,
    num_workers=0,
    collate_fn=_collate_fn)

valid_dl = torch.utils.data.DataLoader(
    dataset=valid_ds,
    batch_size=batch_size,
    shuffle=True,
    num_workers=0,
    collate_fn=_collate_fn)

print(f'train_ds.__len__():{train_ds.__len__()}')
print(f'valid_ds.__len__():{valid_ds.__len__()}')

# # 以下，検証
# _ds = train_ds
# for N in range(2):
#     inp, tgt = _ds.__getitem__(N)
#     print(f'_ds.dataset.ids2inp(inp):{_ds.dataset.ids2inp(inp)}',
#           f'{inp.cpu().numpy()}',
#           f'_ds.dataset.target_ids2target(tgt):{_ds.dataset.target_ids2target(tgt)}',
#           f'{tgt.cpu().numpy()}')

# psylex71_ds.maxlen_out

# 5. 学習

In [ ]:
def fit_a_epoch(model:torch.nn.Module=None,
                optimizer:torch.optim=None,
                loss_f:torch.nn.modules=None,
                _dl:torch.utils.data.dataloader.DataLoader=None):

    model.train()  # モデルを訓練モードに変更

    sum_loss=0
    count=0
    N = 0

    for inps, tchs in _dl:
        inps = pad_sequence(inps, batch_first=True).to(device)
        tchs = pad_sequence(tchs, batch_first=True).to(device)
        outs = model(inps, tchs)

        # 正解のカウント
        out_ids = [out.argmax(dim=1) for out in outs]
        for tch, out in zip(tchs[:], out_ids[:]):
            yesno = ((tch==out) * 1).sum().cpu().numpy() == len(tch)
            count += 1 if yesno else 0

        # 学習の実行
        loss = 0.
        for j in range(len(tchs)):
            loss += loss_f(outs[j],tchs[j])
        loss.backward()  # 損失値の計算
        optimizer.step() # 学習
        sum_loss += loss.item()

        N += len(tchs)
    p_ = count / N
    return model, {'sum_loss':sum_loss, 'count':count, 'N':N, 'P':p_}

In [ ]:
ds = train_ds
n_layers=1
bidirectional=False
n_hid=512
input_tokenizer = gakushu_tokenizer
output_tokenizer = mora_tokenizer

tla_vanilla = vanilla_TLA(inp_vocab_size=len(input_tokenizer.tokens),
                          inp_len=2,
                          out_vocab_size=len(output_tokenizer.tokens),
                          out_len=ds.dataset.maxlen_out,
                          device=device,
                          n_hid=n_hid).to(device)
print(tla_vanilla.eval())

tla_seq2seq = Seq2Seq_wAtt(enc_vocab_size=len(input_tokenizer.tokens),
                           dec_vocab_size=len(output_tokenizer.tokens),
                           n_layers=n_layers,
                           bidirectional=bidirectional,
                           n_hid=n_hid).to(device)
print(tla_seq2seq.eval())

tla_seq2seq0 = Seq2Seq_woAtt(enc_vocab_size=len(input_tokenizer.tokens),
                             dec_vocab_size=len(output_tokenizer.tokens),
                             n_layers=n_layers,
                             bidirectional=bidirectional,
                             n_hid=n_hid).to(device)
print(tla_seq2seq0.eval())

In [ ]:
model0 = tla_vanilla
model1 = tla_seq2seq
model2 = tla_seq2seq0

loss_f = torch.nn.CrossEntropyLoss(ignore_index=-1)
optimizer0 = torch.optim.Adam(model0.parameters(), lr=1e-3)
optimizer1 = torch.optim.Adam(model1.parameters(), lr=1e-3)
optimizer2 = torch.optim.Adam(model2.parameters(), lr=1e-3)
# optimizer0 = torch.optim.Adam(model0.parameters(), lr=1e-5)
# optimizer1 = torch.optim.Adam(model1.parameters(), lr=1e-5)
# optimizer2 = torch.optim.Adam(model2.parameters(), lr=1e-5)

epochs = 500
epochs = 10
#epochs = 200
interval = 3
interval = 1
#interval = 10

for epoch in range(epochs):
    for (model, optimizer) in [(model0,optimizer0), (model1,optimizer1), (model2,optimizer2)]:
    #for (model, optimizer) in [(model2,optimizer2), (model1,optimizer1), (model0,optimizer0)]:
        model, out = fit_a_epoch(model=model, _dl=train_dl, loss_f=loss_f, optimizer=optimizer)
        if (epoch % interval) == 0:
            print(f"エポック:{epoch+1:3d}",
                  f"損失値={out['sum_loss']:10.3f}",
                  f"正解率={out['P']:5.3f}",
                  f"({out['count']:5d}/{out['N']:5d})",
                  end="\t")
    if (epoch % interval) == 0:
        print()

In [ ]:
model = tla_seq2seq
with torch.no_grad():
    for i in range(_ds.__len__()):
        inp, tgt = _ds.__getitem__(i)
        # print(f'インプット:{"".join(c for c in chihaya_ds.ids2tkn(inp))}') #i].cpu().numpy()))}')
        # print(f'ターゲット:{"".join(c for c in chihaya_ds.ids2tkn(tgt))}') #n(c for c in chihaya_ds.ids2tkn(tgt_ids[i].cpu().numpy()))}')

        inp_ids = pad_sequence(inp.unsqueeze(0), batch_first=True).to(device)
        tgt_ids = pad_sequence(tgt.unsqueeze(0), batch_first=True).to(device)

        enc_emb = model.encoder_emb(inp_ids)
        enc_out, (hnx, cnx) = model.encoder(enc_emb)

        dec_inp = torch.tensor([_ds.output_tokenizer.tokens.index('<SOW>')], device=device)
        dec_inp = model.decoder_emb(dec_inp).unsqueeze(0)
        dec_state = (hnx, cnx)

        print(dec_inp.size(), dec_state[0].size())

        for i in range(len(tgt_ids[0])):
            print(f'i:{i}', f'dec_inp.size():{dec_inp.size()}', f'tgt_ids:{tgt_ids}')

            dec_out, dec_state = model.decoder(dec_inp, dec_state)
            dec_inp = dec_out.argmax().unsqueeze(0) # .clone().detach()
            #print(dec_inp, tgt_ids, len(tgt_ids))



        #print(inp_ids.size(), tgt_ids.size())
        #sys.exit()
        #model.encoder(inp_ids, tgt_ids)
        sys.exit()

In [ ]:
model = tla_seq2seq
with torch.no_grad():
    for i in range(_ds.__len__()):
        inp, tgt = chihaya_ds.__getitem__(i)
        # print(f'インプット:{"".join(c for c in chihaya_ds.ids2tkn(inp))}') #i].cpu().numpy()))}')
        # print(f'ターゲット:{"".join(c for c in chihaya_ds.ids2tkn(tgt))}') #n(c for c in chihaya_ds.ids2tkn(tgt_ids[i].cpu().numpy()))}')

        inp_ids = pad_sequence(inp.unsqueeze(0), batch_first=True).to(device)
        tgt_ids = pad_sequence(tgt.unsqueeze(0), batch_first=True).to(device)
        #inp_ids = torch.as_tensor(inp, device=device)
        #tgt_ids = torch.as_tensor(tgt, device=device)
        enc_out, enc_state = model.encoder(inp_ids)
        #dec_out, dec_state = decoder(tgt_ids, enc_state)

        dec_ids = dec_out.argmax(dim=1).detach().cpu().numpy()
        print(f'len(dec_ids):{len(dec_ids)}')
        print("".join(c for c in chihaya_ds.ids2tkn(dec_ids)))

        dec_inp = torch.tensor([chihaya_ds.chihaya_tokens.index('<SOS>')], device=device)
        dec_inp = tgt_ids[0].unsqueeze(0)
        for i in range(len(dec_ids)):
            dec_out, dec_state = decoder(dec_inp, dec_state)
            #print(dec_out.size(), dec_out.argmax().cpu().numpy(), type(dec_out.argmax())) #, dec_out)

            if teacher_forcing:
                dec_inp = tgt_ids[i].unsqueeze(0)
            else:
                dec_inp = dec_out.argmax().unsqueeze(0) # .clone().detach()

            #dec_inp = dec_out.argmax().unsqueeze(0).clone().detach()
            print(f'({dec_inp.cpu().numpy()}',
                  f'{chihaya_ds.chihaya_tokens[dec_inp.cpu().numpy()[0]]})', end=" ") # , type(dec_inp)) # , dec_inp)
            #print(dec_inp.size(), dec_inp.argmax().cpu().numpy(), type(dec_inp.argmax())) # , dec_inp)
            #sys.exit()
        sys.exit()


# Jalex の読み込み

In [ ]:
import pandas as pd
try:
    import jaconv
except:
    !pip install jaconv --upgrade
    import jaconv
# Mecab を使ってヨミを得るために MeCab を import する
from ccap.mecab_settings import wakati, yomi #, parser

jalex_base = os.path.join(HOME, 'study/2025_2014jalex')
jalex_xls_fname = 'JALEX.xlsx'
jalex_fname = os.path.join(jalex_base, jalex_xls_fname)
jalex_DF = pd.read_excel(jalex_fname)
jalex_DF
jalex_words = jalex_DF['目標語']
print(len(jalex_words))

jalex_dic = OrderedDict()
for wrd in jalex_words:
    if not wrd in jalex_dic:
        _yomi = yomi(wrd).strip()
        _wakati = wakati(wrd).strip()
        _mora = mora_tokenizer.wakachi(_yomi)
        _kunrei = kunrei_tokenizer.wakachi(_yomi)
        _hira_yomi = jaconv.kata2hira(_yomi)
        _julius = jaconv.hiragana2julius(_hira_yomi).split(' ')

        jalex_dic[wrd] = {'ヨミ':_yomi, 'モーラ':_mora, '訓令':_kunrei, 'ユリウス':_julius} # , 'Jalex':jalex_DF['wrd']}
        print(jalex_dic[wrd])
        sys.exit()